In [598]:
import pandas as pd
import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

In [573]:
data_set_train=pd.DataFrame(pd.read_csv("train.csv"))
data_set_test=pd.DataFrame(pd.read_csv("test.csv"))


# transformo i valori nelle colonne status,gender e workex in 0 e 1

In [574]:
data_set_train['status']=pd.Categorical( data_set_train["status"]).codes
data_set_train['gender']=pd.Categorical( data_set_train["gender"]).codes
data_set_train['workex']=pd.Categorical( data_set_train["workex"]).codes

data_set_test['status']=pd.Categorical( data_set_test["status"]).codes
data_set_test['gender']=pd.Categorical( data_set_test["gender"]).codes
data_set_test['workex']=pd.Categorical( data_set_test["workex"]).codes

# transformo i valori di queste colonne tramite il OneHot-encoding

In [575]:
data_set_train=pd.get_dummies(data_set_train,columns=['ssc_b','hsc_b','hsc_s','degree_t','specialisation'])
data_set_test=pd.get_dummies(data_set_test,columns=['ssc_b','hsc_b','hsc_s','degree_t','specialisation'])
 


In [576]:
for col in data_set_train.columns:
    if (col in data_set_test.columns)==False:
        data_set_test[col]=0
        

# Nel dataset ci sono dei valori mancanti, sostituisco questi valori mancanti inserendo la media della colonna

In [577]:
data_train_replace_with=data_set_train.mean()
data_set_train=data_set_train.fillna(data_train_replace_with)
data_test_replace_with=data_set_test.mean()
data_set_test=data_set_test.fillna(data_test_replace_with)




In [578]:
data_set_train=data_set_train.reindex(sorted(data_set_train.columns), axis=1)
data_set_test=data_set_test.reindex(sorted(data_set_test.columns), axis=1)

# Uso il columnTranformer con MinMaxScaler per scalare le feature tra 0 e 1


In [579]:
data_set_train.describe()

,degree_p,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,etest_p,gender,hsc_b_Central,hsc_b_Others,hsc_p,hsc_s_Arts,...,hsc_s_Science,mba_p,salary,specialisation_Mkt&Fin,specialisation_Mkt&HR,ssc_b_Central,ssc_b_Others,ssc_p,status,workex
count,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000
mean,65.405194,0.635659,0.069767,0.294574,70.653721,0.635659,0.348837,0.651163,65.118295,0.085271,...,0.426357,62.047209,284512.195122,0.550388,0.449612,0.519380,0.480620,65.927054,0.635659,0.302326
std,7.620323,0.483121,0.255748,0.457628,12.842482,0.483121,0.478460,0.478460,11.634924,0.280374,...,0.496475,5.881485,60800.421964,0.499394,0.499394,0.501572,0.501572,10.545716,0.483121,0.461056
min,50.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,37.000000,0.000000,...,0.000000,51.210000,200000.000000,0.000000,0.000000,0.000000,0.000000,40.890000,0.000000,0.000000
25%,60.000000,0.000000,0.000000,0.000000,60.000000,0.000000,0.000000,0.000000,60.000000,0.000000,...,0.000000,57.690000,250000.000000,0.000000,0.000000,0.000000,0.000000,59.960000,0.000000,0.000000
50%,65.000000,1.000000,0.000000,0.000000,68.000000,1.000000,0.000000,1.000000,64.000000,0.000000,...,0.000000,61.580000,284512.195122,1.000000,0.000000,1.000000,0.000000,66.000000,1.000000,0.000000
75%,70.000000,1.000000,0.000000,1.000000,80.000000,1.000000,1.000000,1.000000,70.400000,0.000000,...,1.000000,65.690000,284512.195122,1.000000,1.000000,1.000000,1.000000,74.000000,1.000000,1.000000
max,91.000000,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,97.700000,1.000000,...,1.000000,77.890000,650000.000000,1.000000,1.000000,1.000000,1.000000,87.000000,1.000000,1.000000


In [580]:

transform=[
    ['min_max_scaler',MinMaxScaler(),['ssc_p','hsc_p','degree_p','etest_p','mba_p','salary']]
]

In [581]:
colum_transf_min_max=ColumnTransformer(transform,remainder='passthrough')

In [582]:
data_set_train=pd.DataFrame(colum_transf_min_max.fit_transform(data_set_train))
data_set_test=pd.DataFrame(colum_transf_min_max.fit_transform(data_set_test))


# Aggiungo i nomi alle colonne 

In [583]:
data_set_train.columns=['ssc_p','hsc_p','degree_p','etest_p','mba_p','salary','degree_t_Comm&Mgmt',
 'degree_t_Others',
 'degree_t_Sci&Tech',
 'gender',
 'hsc_b_Central',
 'hsc_b_Others',
 'hsc_s_Arts',
 'hsc_s_Commerce',
 'hsc_s_Science',
 'specialisation_Mkt&Fin',
 'specialisation_Mkt&HR',
 'ssc_b_Central',
 'ssc_b_Others',
 'status',
 'workex']
data_set_test.columns=['ssc_p','hsc_p','degree_p','etest_p','mba_p','salary','degree_t_Comm&Mgmt',
 'degree_t_Others',
 'degree_t_Sci&Tech',
 'gender',
 'hsc_b_Central',
 'hsc_b_Others',
 'hsc_s_Arts',
 'hsc_s_Commerce',
 'hsc_s_Science',
 'specialisation_Mkt&Fin',
 'specialisation_Mkt&HR',
 'ssc_b_Central',
 'ssc_b_Others',
 'status',
 'workex']


# elimino la colonna Target

In [584]:
Y_train=pd.DataFrame(data_set_train["status"])
Y_test=pd.DataFrame(data_set_test["status"])


In [585]:
data_set_train=data_set_train.drop(['status'],axis=1)
data_set_test=data_set_test.drop(['status'],axis=1)

# Addestro il modello

In [604]:
svm=LinearSVC()

In [605]:
svm.fit(data_set_train,Y_train.values.ravel())

LinearSVC()

In [606]:
predictTest=svm.predict(data_set_test)
predictTrain=svm.predict(data_set_train)

In [607]:
accuracy_test=accuracy_score(predictTest,Y_test)
accuracy_train=accuracy_score(predictTrain,Y_train)

In [608]:
print (f'{accuracy_train}, {accuracy_test}' )

0.8914728682170543, 0.9069767441860465
